# Module 05: Execute queries in Azure Cosmos DB SQL API

- [[Learning path]](https://docs.microsoft.com/en-us/learn/paths/execute-queries-azure-cosmos-db-sql-api/?ns-enrollment-type=Collection&ns-enrollment-id=1k8wcz8zooj2nx)
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/09-sdk-queries.html): Execute a query with the Azure Cosmos DB SQL API SDK
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/10-sdk-pagination.html): Paginate cross-product query results with the Azure Cosmos DB SQL API SDK

## Demo setup

In [ ]:
Connect-AzAccount
Set-AzContext -Subscription "b895a719-7034-411a-9944-ff196d1f450f"
$connectionString = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "ConnectionStrings")["Primary SQL Connection String"]
$primaryMasterKey = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "Keys")["PrimaryMasterKey"]
$documentEndpoint = (Get-AzCosmosDBAccount -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned).DocumentEndpoint

In [ ]:
cosmicworks --endpoint $documentEndpoint --key $primaryMasterKey --datasets product

In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: Microsoft.Azure.Cosmos , 3.22.1"

#!share --from pwsh connectionString
#!share --from pwsh primaryMasterKey
#!share --from pwsh documentEndpoint

In [ ]:
using Microsoft.Azure.Cosmos;
using System;
using System.Collections.Generic;

CosmosClient client = new (connectionString);
Database database = client.GetDatabase("cosmicworks");
Container container = database.GetContainer("products");

public class Product
{
	public string id { get; set; }
	public string categoryId { get; set; }
	public string categoryName { get; set; }
	public string sku { get; set; }
	public string name { get; set; }
	public string description { get; set; }
	public double price { get; set; }
}

### Understand SQL query language

Here is an example of a JSON object that would be in this container:

```json
{
    "id": "86FD9250-4BD5-42D2-B941-1C1865A6A65E",
    "categoryId": "F3FBB167-11D8-41E4-84B4-5AAA92B1E737",
    "categoryName": "Components, Touring Frames",
    "sku": "FR-T67U-58",
    "name": "LL Touring Frame - Blue, 58",
    "description": "The product called \"LL Touring Frame - Blue, 58\"",
    "price": 333.42,
    "tags": [
        {
            "id": "764C1CC8-2E5F-4EF5-83F6-8FF7441290B3",
            "name": "Tag-190"
        },
        {
            "id": "765EF7D7-331C-42C0-BF23-A3022A723BF7",
            "name": "Tag-191"
        }
}
```

### Create queries with SQL

In [ ]:
var sql = "SELECT TOP 10 * FROM products";
QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<Product>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (Product product in currentResultSet)
    {
        Console.WriteLine($"[{product.id}]\t{product.name,35}\t{product.price,15:C}");
    }
}

In [ ]:
// Here is another query that returns only a few fields from a container:
var sql = @"SELECT TOP 10 products.id,
                          products.name,
                          products.price,
                          products.categoryName 
            FROM products";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<Product>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (Product product in currentResultSet)
    {
        Console.WriteLine($"[{product.id}]\t{product.name,35}\t{product.price,15:C}\t{product.categoryName}");
    }
}

In [ ]:
// It doesn’t matter what name is used here for the source
var sql = @"SELECT TOP 10 p.name, 
                          p.price 
            FROM p";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"[{record.name,40}]\t{record.price,15:C}");
    }
}

In [ ]:
// You can use any word or phrase like you would in developer code
var sql = @"SELECT TOP 10 ReferenceContainerHoweverYouLike.name,
                          ReferenceContainerHoweverYouLike.price 
            FROM ReferenceContainerHoweverYouLike";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"[{record.name,40}]\t{record.price,15:C}");
    }
}

In [ ]:
// We can also filter our queries using the WHERE keyword. In this example, we filter the list of products to those that have a price that is between 
// 50 and 100:
var sql = @"SELECT TOP 10   p.name, 
                            p.categoryName, 
                            p.price 
            FROM products p 
            WHERE p.price >= 50 
                AND p.price <= 100";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"[{record.name,40}]\t{record.price,15:C}\t{record.categoryName}");
    }
}

### Project query results

Azure Cosmos DB SQL API extends SQL to manipulate JSON results.

In [ ]:
// use an alias
var sql = @"SELECT TOP 10 p.categoryName AS category FROM products p";
QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"- {record.category}");
    }
}

In [ ]:
// use DISTINCT
var sql = "SELECT DISTINCT TOP 10 p.categoryName FROM products p";
QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"- {record.categoryName}");
    }
}

In [ ]:
// use VALUE -> returns a List<string>
var sql = "SELECT TOP 10 VALUE p.categoryName FROM products p";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<string>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"- {record}");
    }
}

In [ ]:
// Suppose we have the following ProductAdvertisement class definition.
public class ScannerData { 
    public decimal Price { get; set; } 
}

public class ProductAdvertisement { 
    public string Name { get; set; } 
    public string Category { get; set; } 
    public ScannerData ScannerData { get; set;}      
}

In [ ]:
// scannerData expression added to return the expected class format.
var sql = @"SELECT TOP 10 p.name, 
                          p.categoryName AS category, 
                          { ""price"": p.price } AS scannerData 
            FROM products p 
            WHERE p.price >= 50 AND p.price <= 100";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<ProductAdvertisement>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.Name,40}]\t{p.ScannerData.Price,15:C}");
    }
}

### Implement type-checking in queries

NoSQL is schema-less, the responsibility for type checking will often fall on your queries.

```json
// Let’s assume this is a document on the Product container.
{ 
    "id": "6374995F-9A78-43CD-AE0D-5F6041078140", 
    "categoryid": "3E4CEACD-D007-46EB-82D7-31F6141752B2", 
    "sku": "FR-R38R-60", 
    "name": "LL Road Frame - Red, 60", 
    "price": 337.22 
}
```

In [ ]:
// Note how in the previous document there are no tags properties
var sql = @"SELECT TOP 10 p.id, 
                          IS_DEFINED(p.tags) AS tags_exist,  
                          IS_ARRAY(p.tags) as tags_array,
                          IS_NUMBER(p.price) as price_isnumber,
                          IS_STRING(p.price) as price_isstring
            FROM products p ";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.id,40}]\t{p.tags_exist}\t{p.tags_array}\t{p.price_isnumber}\t{p.price_isstring}");
    }
}

### Use built-in functions

SQL for the Azure Cosmos DB SQL API ships with built-in functions for common tasks in a query.

Here are some examples of these functions:

In [ ]:
// CONCAT
var sql = "SELECT TOP 10 VALUE CONCAT(p.name, ' | ', p.categoryName) FROM products p";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<string>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"- {record}");
    }
}

In [ ]:
// LOWER
var sql = "SELECT TOP 10 VALUE LOWER(p.sku) FROM products p";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<string>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var record in currentResultSet)
    {
        Console.WriteLine($"- {record}");
    }
}

In [ ]:
// RTRIM, LTRIM, LEFT, RIGHT

var sql = @"SELECT TOP 10 
                        LTRIM(RTRIM(p.sku)) as sku, 
                        LEFT(p.name, 10) name 
            FROM products p";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.name,40}]\t{p.sku}");
    }
}

In [ ]:
// CONTAINS

var sql = @"SELECT TOP 10 p.name 
            FROM products p
            WHERE CONTAINS(p.name, ""classic"", true)";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.name,40}]");
    }
}

#### ARRAY_CONTAINS

Following example returns all documents (top 10) that have a tagname value Tag-81 in it
It makes use of ARRAY() to create a new array

```json
"tags": [
    {
        "id": "3A3A99B6-E3BF-46D0-BAD9-F5F4DBB720F4",
        "name": "Tag-70"
    },
    {
        "id": "51CD93BF-098C-4C25-9829-4AD42046D038",
        "name": "Tag-81" // looking for this tag!
    },
    {
        "id": "6C2F05C8-1E61-4912-BE1A-C67A378429BB",
        "name": "Tag-5"
    },
    {
        "id": "B48D6572-67EB-4630-A1DB-AFD4AD7041C9",
        "name": "Tag-100"
    }
]
```

is translated into an array:

```json
[
  "Tag-70",
  "Tag-81",
  "Tag-5",
  "Tag-100"
]
```

In [ ]:
// ARRAY_CONTAINS

var sql = @"SELECT TOP 10 
                p.id,
                ARRAY(SELECT DISTINCT VALUE t.name FROM t IN p.tags) as tagNames
            FROM products p 
            WHERE ARRAY_CONTAINS(ARRAY(SELECT DISTINCT VALUE t.name FROM t IN p.tags), ""Tag-81"", true)";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.id,40}]\t{p.tagNames}");
    }
}

## Author complex queries with the Azure Cosmos DB SQL API

### Create cross-product queries

Unlike a JOIN in a relational database, a JOIN in Azure Cosmos DB SQL API scope is a single item only.  A JOIN creates a cross-product between different sections of a single item.

```json
// Suppose this is a document in our product container.
{ 
    "id": "80D3630F-B661-4FD6-A296-CD03BB7A4A0C", 
    "categoryId": "629A8F3C-CFB0-4347-8DCC-505A4789876B",
    "categoryName": "Clothing, Vests", 
    "sku": "VE-C304-L", 
    "name": "Classic Vest, L", 
    "description": "A worn brown classic", 
    "price": 32.4, 
    "tags": [ 
              { 
                "id": "2CE9DADE-DCAC-436C-9D69-B7C886A01B77", 
                "name": "apparel", "class": "group" 
              }, 
              { 
                "id": "CA170AAD-A5F6-42FF-B115-146FADD87298", 
                "name": "worn", "class": "trade-in" 
              }, 
              { 
                "id": "CA170AAD-A5F6-42FF-B115-146FADD87298", 
                "name": "no-damaged", "class": "trade-in" 
              } 
            ] 
}
```

In [ ]:
// Return the tag name for all tags embedded in the document

var sql = @"SELECT TOP 20 p.id, p.name, t.name AS tag 
            FROM products p 
            JOIN t IN p.tags";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.id}]\t{p.name,40}\t{p.tag}");
    }
}

### Implement correlated subqueries

We can optimize JOIN expressions further by writing subqueries to filter the number of array items we want to include in the cross-product set.

```json
{
    "id": "4DA12D36-495E-4DCA-95B0-F18CAA099779",
    "categoryId": "56400CF3-446D-4C3F-B9B2-68286DA3BB99",
    "categoryName": "Bikes, Mountain Bikes",
    "sku": "BK-M82S-42",
    "name": "Mountain-100 Silver, 42",
    "description": "The product called \"Mountain-100 Silver, 42\"",
    "price": 3399.99,
    "tags": [
        {
            "id": "3C26DF5C-CE21-4EF6-AEE2-E8E1066D06B1",
            "name": "Tag-81" // we want to return this item (no other tags needed)
        },
        {
            "id": "BB35DF88-8BCE-4267-838B-9265BAE64EDF",
            "name": "Tag-160"
        },
        {
            "id": "F629F27D-3301-4906-BE9B-C46D6D6F6141",
            "name": "Tag-65"
        }
    ]
}
```

In [ ]:
// Return the tag name for trade-in tags embedded in the document
var sql = @"
SELECT p.id, 
    p.name, 
    t.name AS tag 
FROM products p 
JOIN (SELECT VALUE t 
      FROM t IN p.tags 
      WHERE t.name = 'Tag-81') AS t";

QueryDefinition query = new (sql);

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.id}]\t{p.name,40}\t{p.tag}");
    }
}      

### Implement variables in queries

Using the QueryDefinition class, we can add query parameters.

In [ ]:
// Suppose we want to run the query below from the SDK, but we would like to send the condition values as parameters
// SELECT p.name, t.name AS tag FROM products p JOIN t IN p.tags WHERE p.price > 700 AND p.price <= 800.

// Let’s define a class for our products.
public class Product 
{ 
    public string id { get; set; } 
    public string name { get; set; } 
    public string tag { get; set;}
    public decimal price { get; set; } 
}

// Let’s create our query definition.
string sql = "SELECT TOP 10 p.id, p.name, t.name AS tag, p.price FROM products p JOIN t IN p.tags WHERE p.price >= @lower AND p.price <= @upper";

QueryDefinition query = new QueryDefinition(sql) 
    .WithParameter("@lower", 700) 
    .WithParameter("@upper", 800);

// Let’s finally execute our query. We will run it inside a foreach loop in case we get multiple documents back

var iterator = container.GetItemQueryIterator<dynamic>(query);

while (iterator.HasMoreResults)
{
    var currentResultSet = await iterator.ReadNextAsync();
    foreach (var p in currentResultSet)
    {
        Console.WriteLine($"[{p.id}]\t{p.name,40}\t{p.tag}\t{p.price,15:C}");
    }
}      

### Paginate query results

The Microsoft.Azure.Cosmos.Container class supports asynchronous streams to iterate over multiple pages of results.

In [ ]:
// Suppose we want to run the query below from the SDK, but we would like to return 5 documents at a time

// Let’s define a class for our products.
public class Product 
{ 
    public string id { get; set; } 
    public string name { get; set; } 
    public string price { get; set; } 
}

// Let’s create our query definition.
string sql = "SELECT TOP 12 * FROM products p WHERE p.price > 500"; 

QueryDefinition query = new (sql);
QueryRequestOptions options = new() { MaxItemCount = 5 };

// Let’s finally execute our query. We will run it inside a foreach and a while loop in case we get multiple documents back
FeedIterator<Product> iterator = container.GetItemQueryIterator<Product>(query, requestOptions: options);
int i = 1;

while(iterator.HasMoreResults) 
{ 
    Console.WriteLine($"Pass: {i++}");
    foreach(Product product in await iterator.ReadNextAsync()) 
    { 
        // Handle individual items 
        Console.WriteLine($"- {product.name}");
    } 
}

## Demo teardown

In [ ]:
await database.DeleteAsync();